# Decision Tree Implementation  

This notebook demonstrates an implementation of a **Decision Tree induction algorithm** for classification tasks. The project is designed to showcase the functionality of decision trees while accommodating both real-valued and nominal features.

---

## Key Features:
1. **Decision Tree Algorithm**:  
   - Implements a decision tree induction algorithm for classification.  
   - Handles both real-valued and nominal features.  
   - Does not handle missing values or regression tasks (real-valued target features).  

2. **Split Criterion**:  
   - Users can choose between **information entropy** or **Gini impurity** as the criterion for determining the best split.

3. **Binary Splits for Real-Valued Features**:  
   - Splits are calculated using the following rule:  
     An instance with a feature value lower than the mean feature value follows the left edge from the split node, while all other instances follow the right edge.  

4. **Dataset Demonstration**:  
   - The algorithm is tested on three datasets:  
     - **Iris dataset**  
     - **Wine dataset**  
     - **Adult dataset** from the **UCI Machine Learning Repository**, formatted appropriately for this implementation.

---

## Mathematical Concepts:

### 1. Entropy Formula:
The entropy measures the amount of impurity or uncertainty in the dataset and is defined as:

$ H(T) = - \sum_{i=1}^{n} p_i \log_2(p_i) $

Where:
- $p_i$ is the probability of each class in the dataset.  

### 2. Information Gain:
Information gain measures the reduction in entropy after a dataset is split based on a feature. It is calculated as:

$ IG(T, X) = H(T) - \sum_{i=1}^{k} \frac{|T_i|}{|T|} H(T_i) $

Where:
- $H(T)$ is the entropy of the original dataset $T$.  
- $T_i$ represents the subsets of $T$ resulting from a split on feature $X$.  
- $|T_i|$ and $|T|$ denote the number of examples in subset $T_i$ and the original dataset $T$, respectively.

### 3. Probability:
The probability $p_i$ used in entropy is calculated as:

$ p_i = \frac{\text{Number of instances in class } i}{\text{Total instances in the dataset}} $

This represents the proportion of examples belonging to each class in the dataset.

---

### Example Usage:
This implementation demonstrates the construction of decision trees and evaluates their performance on common classification datasets. For real-valued features, binary splits are applied using the mean value rule described above.

---

### References:
For more information on decision trees and related calculations, refer to the [Wikipedia article on Information Gain](https://en.wikipedia.org/wiki/Information_gain_(decision_tree)).


In [28]:
from collections import Counter
import pandas as pd
import numpy as np

class DecisionTree:
    # Helper class to create our nodes in the decision tree
    class Node:
        def __init__(self, feature=None, threshold=None, left=None, right=None, *, value=None):
            self.feature = feature
            self.threshold = threshold
            self.left = left
            self.right = right
            self.value = value

        def is_leaf_node(self):
            return self.value is not None
    
    def __init__(self, min_samples_split=2, max_depth=100, n_feats=None):
        self.min_samples_split = min_samples_split
        self.max_depth = max_depth
        self.n_feats = n_feats
        self.root = None

    def fit(self, X, y):
        self.n_feats = X.shape[1] if not self.n_feats else min(self.n_feats, X.shape[1])
        self.root = self._grow_tree(X, y)

    def _grow_tree(self, X, y, depth=0):
        n_labels = len(np.unique(y))
        n_samples, n_features = X.shape

        # Stopping criteria
        if (
            depth >= self.max_depth
            or n_labels == 1
            or n_samples < self.min_samples_split
        ):
            leaf_value = self._most_common_label(y)
            return Node(value=leaf_value)

        feat_idxs = np.random.choice(n_features, self.n_feats, replace=False)

        # Determine the best splitting criteria by calculating the
        # information gain through entropy.
        best_feat, best_thresh = self._best_criteria(X, y, feat_idxs)

        # Recursively run _grow_tree on the children as well
        left_idxs, right_idxs = self._split(X[:, best_feat], best_thresh)
        left = self._grow_tree(X[left_idxs, :], y[left_idxs], depth + 1)
        right = self._grow_tree(X[right_idxs, :], y[right_idxs], depth + 1)
        
        return Node(best_feat, best_thresh, left, right)

    def _best_criteria(self, X, y, feat_idxs):
        split_idx, split_thresh = None, None
        best_gain = 0
        
        for feat_idx in feat_idxs:
            X_column = X[:, feat_idx]

            # An instance with a feature value lower than the mean feature value 
            # follows the left edge from the split node while all other instances follow
            # the right edge from the split node. To do this, we first check whether the
            # feature column contains only numbers. If it does, we calculate the mean
            # and set it as the splitting threshold. The other case is that
            # the feature contains some nominal values and we can only separate them
            # by taking their unique values as thresholds.
            if np.issubdtype(X_column.dtype, np.number):
                thresholds = np.mean(X_column)
            else:
                thresholds = np.unique(X_column)            
            
            # Calculate the information gain for each threshold
            for threshold in thresholds:
                gain = self._information_gain(y, X_column, threshold)

                if gain > best_gain:
                    best_gain = gain
                    split_idx = feat_idx
                    split_thresh = threshold

        return split_idx, split_thresh

    def _information_gain(self, y, X_column, split_thresh):
        # parent loss
        parent_entropy = entropy(y)

        # generate split
        left_idxs, right_idxs = self._split(X_column, split_thresh)

        if len(left_idxs) == 0 or len(right_idxs) == 0:
            return 0

        # compute the weighted avg. of the loss for the children
        n = len(y)
        n_left, n_right = len(left_idxs), len(right_idxs)
        e_left, e_right = entropy(y[left_idxs]), entropy(y[right_idxs])
        child_entropy = (n_left / n) * e_left + (n_right / n) * e_right

        # The information gained is the difference in loss before and after split
        info_gain = parent_entropy - child_entropy
        return info_gain

    def _split(self, X_column, split_thresh):
        left_idxs = np.argwhere(X_column <= split_thresh).flatten()
#         print(X_column[left_idxs],X_column[left_idxs])
        right_idxs = np.argwhere(X_column > split_thresh).flatten()
        return left_idxs, right_idxs

    def _traverse_tree(self, x, node):
        if node.is_leaf_node():
            return node.value

        if x[node.feature] <= node.threshold:
            return self._traverse_tree(x, node.left)
        return self._traverse_tree(x, node.right)

    def _most_common_label(self, y):
        counter = Counter(y)
        most_common = counter.most_common(1)[0][0]
        return most_common
    
    def predict(self, X):
        return np.array([self._traverse_tree(x, self.root) for x in X])
    
def entropy(y):
    hist = np.bincount(y)
    ps = hist / len(y)
    return -np.sum([p * np.log2(p) for p in ps if p > 0])

def accuracy(y_true, y_pred):
    accuracy = np.sum(y_true == y_pred) / len(y_true)
    return accuracy

def train_test_split(X, y, test_size=0.2, random_state=None):
    np.random.seed(random_state)
    shuffle_index = np.random.permutation(len(X))
    test_index = int(len(X) * test_size)
    X_train = X[shuffle_index[test_index:]]
    y_train = y[shuffle_index[test_index:]]
    X_test = X[shuffle_index[:test_index]]
    y_test = y[shuffle_index[:test_index]]
    return X_train, X_test, y_train, y_test 

## Iris Dataset

In [29]:
filename = 'iris.tmls'
data = pd.read_csv(filename)
# Remove the line with "r,r,r,r,n"
data = data.drop(data.index[0])

y_data = data['class']
# Replace class names
for i, value in enumerate(np.unique(y_data)):
    y_data = np.where(y_data == value, i, y_data)
y_data = np.asarray(y_data)
y_data = y_data.astype(dtype=np.int64)
data

,sepal length,sepal width,petal length,petal width,class
1,5.1,3.5,1.4,0.2,Iris-setosa
2,4.9,3.0,1.4,0.2,Iris-setosa
3,4.7,3.2,1.3,0.2,Iris-setosa
4,4.6,3.1,1.5,0.2,Iris-setosa
5,5.0,3.6,1.4,0.2,Iris-setosa
...,...,...,...,...,...
146,6.7,3.0,5.2,2.3,Iris-virginica
147,6.3,2.5,5.0,1.9,Iris-virginica
148,6.5,3.0,5.2,2.0,Iris-virginica
149,6.2,3.4,5.4,2.3,Iris-virginica


In [30]:
print("Iris Dataset - Multiple runs:\n")
for i in range(5):
    x_data = data.iloc[:, :-1].values
    X_train, X_test, y_train, y_test = train_test_split(
        x_data, y_data, test_size=0.2, random_state=None
    )

    dt = DecisionTree(max_depth=10)
    dt.fit(X_train, y_train)

    y_pred = dt.predict(X_test)

    acc = accuracy(y_test, y_pred)

    print("Accuracy:", acc)


Iris Dataset - Multiple runs:

Accuracy: 0.8666666666666667
Accuracy: 0.9333333333333333
Accuracy: 0.9
Accuracy: 1.0
Accuracy: 0.9


## Wine Dataset

In [31]:
filename = 'wine.tmls'
data = pd.read_csv(filename)
# Remove the line with "r,r,r,r,r,r,r,r,r,r,r,r,r,r"
data = data.drop(data.index[0])

y_data = data['class']
# Replace class names
for i, value in enumerate(np.unique(y_data)):
    y_data = np.where(y_data == value, i, y_data)
y_data = np.asarray(y_data)
y_data = y_data.astype(dtype=np.int64)
data

,alcohol,malic acid,ash,alcalinity of ash,magnesium,total phenols,flavanoids,nonflavanoid phenols,proanthocyanins,color intensity,hue,OD280/OD315 of diluted wines,proline,class
1,14.23,1.71,2.43,15.6,127,2.8,3.06,.28,2.29,5.64,1.04,3.92,1065,1
2,13.2,1.78,2.14,11.2,100,2.65,2.76,.26,1.28,4.38,1.05,3.4,1050,1
3,13.16,2.36,2.67,18.6,101,2.8,3.24,.3,2.81,5.68,1.03,3.17,1185,1
4,14.37,1.95,2.5,16.8,113,3.85,3.49,.24,2.18,7.8,.86,3.45,1480,1
5,13.24,2.59,2.87,21,118,2.8,2.69,.39,1.82,4.32,1.04,2.93,735,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
174,13.71,5.65,2.45,20.5,95,1.68,.61,.52,1.06,7.7,.64,1.74,740,3
175,13.4,3.91,2.48,23,102,1.8,.75,.43,1.41,7.3,.7,1.56,750,3
176,13.27,4.28,2.26,20,120,1.59,.69,.43,1.35,10.2,.59,1.56,835,3
177,13.17,2.59,2.37,20,120,1.65,.68,.53,1.46,9.3,.6,1.62,840,3


In [32]:
print("Wine Dataset - Multiple runs:\n")
for i in range(5):
    x_data = data.iloc[:, :-1].values
    X_train, X_test, y_train, y_test = train_test_split(
        x_data, y_data, test_size=0.2, random_state=None
    )

    dt = DecisionTree(max_depth=10)
    dt.fit(X_train, y_train)

    y_pred = dt.predict(X_test)

    acc = accuracy(y_test, y_pred)

    print("Accuracy:", acc)

Wine Dataset - Multiple runs:

Accuracy: 0.8285714285714286
Accuracy: 0.8285714285714286
Accuracy: 0.8571428571428571
Accuracy: 0.8571428571428571
Accuracy: 0.8857142857142857


## Adult Dataset

In [33]:
filename = 'adult.tmls'
data = pd.read_csv(filename)
# Remove the line with "r,n,r,n,r,n,n,n,r,r,r,n"
data = data.drop(data.index[0])

y_data = data['class']
# Replace class names
for i, value in enumerate(np.unique(y_data)):
    y_data = np.where(y_data == value, i, y_data)
y_data = np.asarray(y_data)
y_data = y_data.astype(dtype=np.int64)
data

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,class
1,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40.0,United-States,<=50K
2,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13.0,United-States,<=50K
3,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40.0,United-States,<=50K
4,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40.0,United-States,<=50K
5,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40.0,Cuba,<=50K
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32557,27,Private,257302,Assoc-acdm,12,Married-civ-spouse,Tech-support,Wife,White,Female,0,0,38.0,United-States,<=50K
32558,40,Private,154374,HS-grad,9,Married-civ-spouse,Machine-op-inspct,Husband,White,Male,0,0,40.0,United-States,>50K
32559,58,Private,151910,HS-grad,9,Widowed,Adm-clerical,Unmarried,White,Female,0,0,40.0,United-States,<=50K
32560,22,Private,201490,HS-grad,9,Never-married,Adm-clerical,Own-child,White,Male,0,0,20.0,United-States,<=50K


In [34]:
print("Adult Dataset - Multiple runs:\n[Warning] This takes a while.\n")
    
for i in range(5):
    x_data = data.iloc[:, :-1].values
    X_train, X_test, y_train, y_test = train_test_split(
        x_data, y_data, test_size=0.2, random_state=None
    )

    dt = DecisionTree(max_depth=10)
    dt.fit(X_train, y_train)

    y_pred = dt.predict(X_test)

    acc = accuracy(y_test, y_pred)

    print("Accuracy:", acc)


Adult Dataset - Multiple runs:
[Warning] This takes a while.

Accuracy: 0.8482800982800983
Accuracy: 0.8527334152334153
Accuracy: 0.8493550368550369
Accuracy: 0.8490479115479116
Accuracy: 0.847512285012285
